In [161]:
import pandas as pd
import datetime

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [162]:
merged1 = pd.read_csv('new_data/merged_clean.csv', index_col=0)
print(merged1.shape)
merged2 = pd.read_csv('new_data/merged_clean_2.csv', index_col=0)
print(merged2.shape)
merged = merged1.append(merged2)
print(merged.shape)

(271100, 21)
(207447, 21)
(478547, 21)


/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/3283179314.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged = merged1.append(merged2)


In [163]:
#drop duplicates
merged.drop_duplicates(inplace=True)

#drop all na
merged.dropna(subset=['ProfileScore:', 'Vert. meters:', 'Distance:'], how='all', inplace=True)
#merged.dropna(subset=['ProfileScore:', 'Vert. meters:', 'Distance:'], how='all', inplace=True)
merged.reset_index(inplace=True, drop=True)
merged.shape

(388692, 21)

In [164]:
merged.columns

Index(['name', 'year', 'type', 'date', 'result', 'gc', 'icon', 'race_ref',
       'race_name', 'race_detail', 'race_rank', 'distance', 'points',
       'Avg. speed winner:', 'Distance:', 'Parcours type:', 'ProfileScore:',
       'Vert. meters:', 'Startlist quality score:', 'Won how: ',
       'adjusted_points'],
      dtype='object')

In [165]:
merged['ProfileScore:'].value_counts()

0.0      13766
1.0       3887
5.0       3050
6.0       2762
9.0       2522
         ...  
205.0        5
538.0        4
289.0        1
259.0        1
335.0        1
Name: ProfileScore:, Length: 403, dtype: int64

In [166]:
#set date as datetime
merged['date'] = pd.to_datetime(merged['date'])

### Vertical meters

In [167]:
no_na = merged.dropna(subset=['ProfileScore:', 'Vert. meters:', 'distance'], how='any')

In [168]:
# profile score via vert and distance
X  = no_na[['Distance:', 'ProfileScore:']]
y = no_na['Vert. meters:']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

tree = DecisionTreeRegressor()

tree.fit(X_train_scaled, y_train)

tree.score(X_test_scaled, y_test)

0.999732838795519

In [169]:
vert_na = merged[merged['Vert. meters:'].isna()]
vert_na.dropna(subset=['Distance:', 'ProfileScore:'], how='any', inplace=True)

X_pred = vert_na[['Distance:', 'ProfileScore:']]

X_pred_scaled = scaler.transform(X_pred)

vert_na['predicted_vert'] = tree.predict(X_pred_scaled)
vert_na

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/1125638154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vert_na.dropna(subset=['Distance:', 'ProfileScore:'], how='any', inplace=True)
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/1125638154.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vert_na['predicted_vert'] = tree.predict(X_pred_scaled)


,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points,predicted_vert
5605,richard-carapaz,2020,etappe,2020-08-09,0,NaN,stage,race/tour-de-pologne/2020/stage-5,tour-de-pologne,Stage 5 - Zakopane › Kraków,...,0.0,41.57,188.0,p2,45.0,NaN,663,Sprint of large group,0.0,1346.0
5606,luke-rowe,2020,etappe,2020-08-09,109,NaN,stage,race/tour-de-pologne/2020/stage-5,tour-de-pologne,Stage 5 - Zakopane › Kraków,...,0.0,41.57,188.0,p2,45.0,NaN,663,Sprint of large group,0.0,1346.0
5607,maximilian-schachmann,2020,etappe,2020-08-09,24,NaN,stage,race/tour-de-pologne/2020/stage-5,tour-de-pologne,Stage 5 - Zakopane › Kraków,...,0.0,41.57,188.0,p2,45.0,NaN,663,Sprint of large group,0.0,1346.0
5608,dries-devenyns,2020,etappe,2020-08-09,77,NaN,stage,race/tour-de-pologne/2020/stage-5,tour-de-pologne,Stage 5 - Zakopane › Kraków,...,0.0,41.57,188.0,p2,45.0,NaN,663,Sprint of large group,0.0,1346.0
5609,rudy-molard,2020,etappe,2020-08-09,50,NaN,stage,race/tour-de-pologne/2020/stage-5,tour-de-pologne,Stage 5 - Zakopane › Kraków,...,0.0,41.57,188.0,p2,45.0,NaN,663,Sprint of large group,0.0,1346.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280443,ivan-basso,2013,etappe,2013-04-19,17,NaN,stage,race/tour-of-the-alps/2013/stage-4,tour-of-the-alps,Stage 4 - Arco › Sega di Ala,...,8.0,38.23,166.8,p5,329.0,NaN,420,? - let us know!,1105440.0,4634.0
280444,leopold-konig,2013,etappe,2013-04-19,20,NaN,stage,race/tour-of-the-alps/2013/stage-4,tour-of-the-alps,Stage 4 - Arco › Sega di Ala,...,2.0,38.23,166.8,p5,329.0,NaN,420,? - let us know!,276360.0,4634.0
280445,jan-barta,2013,etappe,2013-04-19,58,NaN,stage,race/tour-of-the-alps/2013/stage-4,tour-of-the-alps,Stage 4 - Arco › Sega di Ala,...,0.0,38.23,166.8,p5,329.0,NaN,420,? - let us know!,0.0,4634.0
280446,david-de-la-cruz,2013,etappe,2013-04-19,24,NaN,stage,race/tour-of-the-alps/2013/stage-4,tour-of-the-alps,Stage 4 - Arco › Sega di Ala,...,0.0,38.23,166.8,p5,329.0,NaN,420,? - let us know!,0.0,4634.0


In [170]:
merged.loc[vert_na.index, 'Vert. meters:'] = vert_na['predicted_vert']

### Decision tree for profile score

In [171]:
#drop where either profile score or vert meters are missing
no_na = merged.dropna(subset=['ProfileScore:', 'Vert. meters:'], how='any')
print(no_na.shape)

#drop where score below 10 -> arbitrary point
no_na = no_na[no_na['ProfileScore:'] > 10]
print(no_na.shape)

(257974, 21)
(220014, 21)


In [172]:
# profile score via vert and distance
X  = no_na[['Distance:', 'Vert. meters:']]
y = no_na['ProfileScore:']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

tree = DecisionTreeRegressor()

tree.fit(X_train_scaled, y_train)

tree.score(X_test_scaled, y_test)

0.9999567649009945

In [173]:
profile_na = merged[merged['ProfileScore:'].isna()]
smaller_ten = merged[merged['ProfileScore:'] <= 10]
impute_profile = profile_na.append(smaller_ten)
impute_profile.dropna(subset=['Distance:', 'Vert. meters:'], how='any', inplace=True)

X_pred = impute_profile[['Distance:', 'Vert. meters:']]

X_pred_scaled = scaler.transform(X_pred)

impute_profile['predicted_score'] = tree.predict(X_pred_scaled)
impute_profile

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/821046524.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  impute_profile = profile_na.append(smaller_ten)


,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points,predicted_score
3648,egan-bernal,2020,etappe,2020-08-01,6,8.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,30.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,49.0
3649,andrey-amador,2020,etappe,2020-08-01,77,77.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,49.0
3650,jonathan-castroviejo,2020,etappe,2020-08-01,75,75.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,49.0
3651,pavel-sivakov,2020,etappe,2020-08-01,40,41.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,49.0
3652,dylan-van-baarle,2020,etappe,2020-08-01,81,81.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380923,marcos-garcia,2010,etappe,2010-08-28,12,79.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,13.0
380924,vladimir-isaychev,2010,etappe,2010-08-28,12,75.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,13.0
380925,serafin-martinez,2010,etappe,2010-08-28,12,80.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,13.0
380926,gonzalo-rabunal-rios,2010,etappe,2010-08-28,12,81.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,13.0


In [174]:
merged.loc[impute_profile.index, 'ProfileScore:'] = impute_profile['predicted_score']

### KNN to impute parcours type

In [175]:
merged['Parcours type:'].value_counts()

p2    103706
p1     84824
p0     70583
p5     56511
p3     45573
p4     27495
Name: Parcours type:, dtype: int64

In [176]:
knn = KNeighborsClassifier(n_neighbors=5)

px0 = merged[merged['Parcours type:']!='p0']

px0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)

X = px0[['ProfileScore:', 'Vert. meters:']]

y = px0['Parcours type:']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

knn.fit(X_train_scaled, y_train)

knn.score(X_test_scaled, y_test)


/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/977234109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  px0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)


0.99148879044167

In [177]:
p0 = merged[merged['Parcours type:']=='p0']

p0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)

X_pred = p0[['ProfileScore:', 'Vert. meters:']]

X_pred_scaled = scaler.transform(X_pred)

labels = knn.predict(X_pred_scaled)

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/1480808305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)


In [178]:
p0['adjusted parcours type'] = labels

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_54260/2894937052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p0['adjusted parcours type'] = labels


In [179]:
p0['adjusted parcours type'].value_counts()

p2    4916
p3    3340
p1    2430
p5    1156
p4     861
Name: adjusted parcours type, dtype: int64

In [180]:
merged.loc[p0.index, 'Parcours type:'] = p0['adjusted parcours type']

In [181]:
merged['Parcours type:'].value_counts()

p2    108622
p1     87254
p0     57880
p5     57667
p3     48913
p4     28356
Name: Parcours type:, dtype: int64

In [146]:
merged.groupby('Parcours type:')[['Vert. meters:', 'ProfileScore:']].mean()

,Vert. meters:,ProfileScore:
Parcours type:,,
p0,NaN,NaN
p1,792.499519,25.000902
p2,2174.695311,86.222025
p3,2373.901675,110.816278
p4,3317.368918,182.551637
p5,3561.326972,280.037628


In [182]:
parcours = {
    'p1' : 'fl',
    'p2' : 'hi_fl',
    'p3' : 'hi_hi',
    'p4' : 'mo_fl',
    'p5' : 'mo_mo'
}

merged['Parcours type:'] = merged['Parcours type:'].map(parcours)

In [184]:
merged.dropna(subset=['ProfileScore:', 'Vert. meters:', 'distance'], how='any', inplace=True)
merged.shape

(261001, 21)

### Test split data

In [205]:
#test data
test_data = merged[(merged['name'] == 'tadej-pogacar') & (merged['year'] == 2021) & (merged['race_name'] == 'tour-de-france')]
test_data_x = test_data[['date', 'distance', 'ProfileScore:', 'Vert. meters:', 'Startlist quality score:']].set_index('date').T
test_data_y = test_data[['date', 'points']].set_index('date').T

In [207]:
test_data_y

date,2021-07-18,2021-07-17,2021-07-16,2021-07-15,2021-07-14,2021-07-13,2021-07-11,2021-07-10,2021-07-09,2021-07-08,...,2021-07-06,2021-07-04,2021-07-03,2021-07-02,2021-07-01,2021-06-30,2021-06-29,2021-06-28,2021-06-27,2021-06-26
points,0.0,26.0,0.0,50.0,50.0,14.0,0.0,6.0,0.0,0.0,...,0.0,30.0,36.0,0.0,0.0,50.0,0.0,0.0,44.0,30.0


In [200]:
#train data
train_data = merged[(merged['name'] == 'tadej-pogacar') & (merged['date'] < min(test_data['date'])) & (merged['date'] >= min(test_data['date']) - datetime.timedelta(weeks=52))]
train_data_x = train_data[['date', 'distance', 'ProfileScore:', 'Vert. meters:', 'Startlist quality score:']]
train_data_y = train_data[['date', 'points']]

In [201]:
train_data_y

,date,points
93,2020-09-16,40.0
248,2020-09-15,0.0
405,2020-09-13,50.0
564,2020-09-12,18.0
724,2020-09-11,16.0
885,2020-09-10,0.0
1047,2020-09-09,0.0
1211,2020-09-08,0.0
1377,2020-09-06,50.0
1547,2020-09-05,24.0


In [191]:
merged[(merged['name'] == 'tadej-pogacar') & (merged['date'] < min(y_test['date'])) & (merged['date'] >= min(y_test['date']) - datetime.timedelta(weeks=52))]

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,distance,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points
93,tadej-pogacar,2020,etappe,2020-09-16,3,2.0,stage,race/tour-de-france/2020/stage-17,tour-de-france,Stage 17 - Grenoble › Méribel - Col de la Loze,...,170.00,40.0,35.280,170.00,mo_mo,444.0,4430.0,1685,3.5 km solo,29925600.0
248,tadej-pogacar,2020,etappe,2020-09-15,21,2.0,stage,race/tour-de-france/2020/stage-16,tour-de-france,Stage 16 - La Tour-du-Pin › Villard-de-Lans,...,164.00,0.0,38.910,164.00,mo_mo,250.0,3903.0,1685,25 km solo,0.0
405,tadej-pogacar,2020,etappe,2020-09-13,1,2.0,stage,race/tour-de-france/2020/stage-15,tour-de-france,Stage 15 - Lyon › Grand Colombier,...,174.50,50.0,38.180,174.50,mo_mo,390.0,3724.0,1685,Sprint a deux,32857500.0
564,tadej-pogacar,2020,etappe,2020-09-12,12,2.0,stage,race/tour-de-france/2020/stage-14,tour-de-france,Stage 14 - Clermont-Ferrand › Lyon,...,194.00,18.0,43.410,194.00,mo_fl,73.0,2646.0,1685,3 km solo,2214090.0
724,tadej-pogacar,2020,etappe,2020-09-11,13,2.0,stage,race/tour-de-france/2020/stage-13,tour-de-france,Stage 13 - Châtel-Guyon › Pas de Peyrol (Le Pu...,...,191.50,16.0,38.070,191.50,mo_mo,302.0,4459.0,1685,0.1 km solo,8141920.0
885,tadej-pogacar,2020,etappe,2020-09-10,26,7.0,stage,race/tour-de-france/2020/stage-12,tour-de-france,Stage 12 - Chauvigny › Sarran,...,218.00,0.0,42.360,218.00,hi_hi,142.0,3389.0,1685,28 km solo,0.0
1047,tadej-pogacar,2020,etappe,2020-09-09,34,7.0,stage,race/tour-de-france/2020/stage-11,tour-de-france,Stage 11 - Châtelaillon-Plage › Poitiers,...,166.50,0.0,41.620,166.50,fl,13.0,1008.0,1685,Sprint of large group,0.0
1211,tadej-pogacar,2020,etappe,2020-09-08,25,7.0,stage,race/tour-de-france/2020/stage-10,tour-de-france,Stage 10 - île d'Oléron (Le Château-d'Oléron) ...,...,168.50,0.0,46.940,168.50,fl,18.0,531.0,1685,Sprint of large group,0.0
1377,tadej-pogacar,2020,etappe,2020-09-06,1,7.0,stage,race/tour-de-france/2020/stage-9,tour-de-france,Stage 9 - Pau › Laruns,...,153.00,50.0,39.020,153.00,mo_fl,239.0,3500.0,1685,Sprint of small group,20135750.0
1547,tadej-pogacar,2020,etappe,2020-09-05,9,9.0,stage,race/tour-de-france/2020/stage-8,tour-de-france,Stage 8 - Cazères › Loudenvielle,...,141.00,24.0,34.930,141.00,mo_fl,295.0,3821.0,1685,35 km solo,11929800.0


In [41]:
min(y_test['date']) - datetime.timedelta(weeks=52)

Timestamp('2020-06-27 00:00:00')

In [189]:
min(y_test['date'])

Timestamp('2021-06-26 00:00:00')